In [76]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

In [77]:
# Fetching the data
def fetch_data():
    end_date = pd.Timestamp.now().date()
    start_date = end_date - pd.DateOffset(years=10)
    nvidia_data = yf.download('NVDA', start=start_date, end=end_date)
    return nvidia_data['Close']

data = fetch_data().values.reshape(-1, 1)

[*********************100%***********************]  1 of 1 completed


In [78]:
# Data Preprocessing
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 10
X, y = create_dataset(data, look_back)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=117)


In [79]:
# Convert data to PyTorch tensors
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

In [80]:
# Reshaping to rows, timestamps, features
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors, (X_test_tensors.shape[0], 1, X_test_tensors.shape[1]))


In [81]:
# Define LSTM Model
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.linear(out[:, -1, :])
        return out

input_dim = look_back  
hidden_dim = 64
num_layers = 2
output_dim = 1

model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


y_train_tensors = y_train_tensors.view(-1, 1)  # Reshape the tensor

epochs = 100

for epoch in range(epochs):
    outputs = model.forward(X_train_tensors_final)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train_tensors)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:  # Print loss every 10 epochs
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 18593.994140625


Epoch 10, Loss: 17624.474609375
Epoch 20, Loss: 16631.279296875
Epoch 30, Loss: 15795.166015625
Epoch 40, Loss: 15048.125
Epoch 50, Loss: 14389.47265625
Epoch 60, Loss: 13813.1064453125
Epoch 70, Loss: 13310.5986328125
Epoch 80, Loss: 12873.8251953125
Epoch 90, Loss: 12495.3828125


In [82]:
look_back = 5
trainX, trainY = create_dataset(data, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
trainX = torch.from_numpy(trainX).type(torch.Tensor)
trainY = torch.from_numpy(trainY).type(torch.Tensor)


In [83]:
# Save Model
torch.save(model.state_dict(), 'lstm_model.pth')